In [48]:
from typing import Callable, Protocol, NamedTuple
from collections import Counter

import torch
from jaxtyping import Int, Float, Bool
from transformers import PreTrainedTokenizer, AutoTokenizer
from transformer_lens import HookedTransformer, HookedTransformerConfig

In [8]:
# get the gpt2 tokenizer
TOKENIZER: PreTrainedTokenizer = AutoTokenizer.from_pretrained("gpt2")

# tokens which have quotes

In [11]:
print(f"{len(TOKENIZER)} ")
tokens_with_quotes: list[str] = [
	x 
    for x in TOKENIZER.get_vocab().keys() 
    if ('"' in x)
]
print(f"{len(tokens_with_quotes) = }")

50257 
len(tokens_with_quotes) = 131


In [12]:
print(tokens_with_quotes)

['{"', 'Ġ"-', 'Ġ"[', '"]=>', 'Ġ"_', '="#', '">', '"""', '="', '?\'"', 'Ġ"#', '\',"', '":"","', '",', '/"', '.""', 'Ġ."', 'Ġ,"', '"/>', '":["', '").', '!"', 'Ġ"""', '"!', 'Ġ"(', 'âĢĶ"', '="/', '"âĢ¦', ';"', 'Ġ"$:/', 'âĢ¦."', '":', '!!"', '"><', '".', '"-', '.",', ',"', 'Ġ["', '%"', 'Ġ("', '=""', '"âĢĶ', '."', '"],"', '\\",', '"},', ']."', '\\">', '?!"', '\'"', '"}],"', ')."', 'Ġ"\'', '"],', '!",', '":-', '?".', 'Ġ..."', '=\\"', 'Ġ"...', ':"', '!".', '("', '},{"', '"))', '!\'"', '"),', ',\'"', 'Ġ".', '")', '":"/', '..."', '."[', '.\'"', 'Ġ"{', '"...', '"},"', 'Ġ"\\', '"></', ']"', '.","', '");', '.,"', '".[', '},"', 'Ġ\\"', '?"', 'Ġ"<', '?",', '\\":', '>"', '\\"', 'Ġ"+', '":[', '"\'', '"},{"', '":{"', '!?"', '),"', '","', '["', 'Ġ{"', '"[', '"]', 'âĢ¦"', 'Ġ"', '":"', 'Ġ"%', '"?', '";', '.")', '":""},{"', 'Ġ""', '}"', 'ĠâĢ¦"', '""', 'Ġ"$', '":[{"', 'Ġ",', '"', '-"', '"(', ')"', 'Ġ"/', '],"', '"}', '\'."', ')",', 'Ġ"âĢ¦', 'Ġ"@']


## now, which of these show up in tinystories?

In [3]:
# load a subset of the tinystories dataset
with open("../data/tiny_stories/tinystories_10k.txt", "r", encoding="utf-8") as f:
    TEXT_DATA: list[str] = f.read().split("<|endoftext|>")

In [27]:
# split into tokens but don't convert to ids
text_data_tokenized = [
	TOKENIZER(x).tokens()
	for x in TEXT_DATA
]

from itertools import chain
text_data_tokenized_joined = list(chain(*text_data_tokenized))

In [29]:
print(f"{len(text_data_tokenized) = }")
print(f"{len(text_data_tokenized_joined) = }")

len(text_data_tokenized) = 1728
len(text_data_tokenized_joined) = 349321


In [34]:
# set of tokens with quotes which appear in the dataset


quote_tokens_in_data = Counter([
	x
	for x in text_data_tokenized_joined
	if ('"' in x)
])


# print(f"{quote_tokens_in_data = }")
for x in quote_tokens_in_data:
	print(f"{quote_tokens_in_data[x]}\t`{x}`")

2983	`Ġ"`
1262	`!"`
1340	`."`
702	`?"`
807	`"`
364	`,"`
41	`".`
3	`?!"`
3	`?".`
2	`'."`
2	`?",`
1	`..."`
3	`",`
5	`!".`
1	`Ġ"'`
1	`',"`


# Creating a feature generator

In [49]:
BinaryFeatureExtractorOutput = NamedTuple(
    "FeatureExtractorOutput",
    [
        ("tokens", list[str]),
        ("input_ids", Int[torch.Tensor, "seq_len"]),
        ("features", Bool[torch.Tensor, "seq_len"]),
    ],
)

FeatureExtractor = Callable[[str, PreTrainedTokenizer], BinaryFeatureExtractorOutput]

class FeatureExtractorConfigurable(Protocol):
    def __call__(
        self,
        text: str,
        tokenizer: PreTrainedTokenizer,
        **kwargs
    ) -> BinaryFeatureExtractorOutput:
        ...

In [50]:
def in_quotes_feature(
        text: str, 
        tokenizer: PreTrainedTokenizer,
        qmark_in_quote: bool = False,
    ) -> BinaryFeatureExtractorOutput:
    # Tokenize the text
    tokens: list[str] = tokenizer.tokenize(text)

    # Convert tokens to IDs
    token_ids: list[int] = tokenizer.convert_tokens_to_ids(tokens)

    # Compute binary feature for each token (1 if inside a quote, 0 otherwise)
    quote_feature: list[bool] = []
    inside_quote: bool = False # assume we arent in a quote to start
    is_qmark: bool = False

    for token in tokens:
        if "\"" in token:
            is_qmark = True
            inside_quote = not inside_quote
        else:
            is_qmark = False

        quote_feature.append(
            qmark_in_quote # tokens with `"` treated separately 
            if is_qmark 
            else inside_quote # otherwise, use the current state
        ) 

    return BinaryFeatureExtractorOutput(
        tokens=tokens,
        input_ids=torch.tensor(token_ids),
        features=torch.tensor(quote_feature),
    )


in_quotes_no_qmark_feature: FeatureExtractor = lambda text, tokenizer: in_quotes_feature(text, tokenizer, qmark_in_quote=False)
in_quotes_with_qmark_feature: FeatureExtractor = lambda text, tokenizer: in_quotes_feature(text, tokenizer, qmark_in_quote=True)


def display_results(text: str, tokenizer: PreTrainedTokenizer, feature_extractor: FeatureExtractor) -> None:
    tokens, _, quote_feature_tensor = feature_extractor(text, tokenizer)

    print("Input text:")
    for token, feature in zip(tokens, quote_feature_tensor):
        if feature == 1:
            # print(f"\033[43m{token}\033[0m", end=" ")
            # dark blue background
            print(f"\033[44m{token}\033[0m", end=" ")
        else:
            print(token, end=" ")
    print("\n")
    
display_results(TEXT_DATA[8], TOKENIZER, in_quotes_no_qmark_feature)
display_results(TEXT_DATA[8], TOKENIZER, in_quotes_with_qmark_feature)

Input text:
ĊĊ Ċ S ara Ġand ĠBen Ġwanted Ġto Ġdecor ate Ġa Ġbowl Ġfor Ġtheir Ġmom . ĠThey Ġfound Ġa Ġbig Ġbowl Ġin Ġthe Ġkitchen Ġand Ġsome Ġpaint Ġand Ġbrushes . ĠThey Ġtook Ġthe Ġbowl Ġand Ġthe Ġpaint Ġto Ġthe Ġbackyard Ġand Ġput Ġthem Ġon Ġa Ġtable . Ċ " Let 's Ġmake Ġthe Ġbowl Ġpretty Ġwith Ġcolors ," ĠSara Ġsaid . Ċ " OK , ĠI Ġwill Ġpaint Ġa Ġflower ," ĠBen Ġsaid . Ċ They Ġstarted Ġto Ġpaint Ġthe Ġbowl Ġwith Ġdifferent Ġcolors . ĠSara Ġpainted Ġa Ġred Ġheart Ġand ĠBen Ġpainted Ġa Ġyellow Ġflower . ĠThey Ġwere Ġhaving Ġfun . Ċ But Ġthen , Ġit Ġstarted Ġto Ġrain . ĠThe Ġrain Ġwas Ġwet Ġand Ġcold . ĠIt Ġmade Ġthe Ġpaint Ġrun Ġand Ġdrip . ĠThe Ġbowl Ġlooked Ġmessy Ġand Ġugly . Ċ " Oh Ġno , Ġthe Ġrain Ġruined Ġour Ġbowl !" ĠSara Ġcried . Ċ " Mom Ġwill Ġnot Ġlike Ġit ," ĠBen Ġsaid . Ċ They Ġran Ġinside Ġthe Ġhouse Ġwith Ġthe Ġbowl . ĠThey Ġwere Ġsad Ġand Ġwet . Ċ They Ġshowed Ġthe Ġbowl Ġto Ġtheir Ġmom . ĠThey Ġsaid Ġthey Ġwere Ġsorry . Ċ But Ġmom Ġsmiled Ġand Ġhugged Ġthem . ĠShe Ġsaid